search jupyter notebooks using llama-gpt?

search llama?



In [ ]:
//import {fileURLToPath} from "url";
const importNotebook = importer.import
const storeLlamaFunction = importNotebook('store llama function')
const {getExports, selectAst, cacheCells} = importNotebook([
    'select code tree', 'get exports from source', 'cache notebook', 'cache all'])
const llama = importNotebook('create llm session')
const {askLlamaAboutNotebooks} = importNotebook('ask llm about notebooks')
const {askLlamaAboutCode} = importNotebook('ask llm about code')
const {askLlamaToSummerize} = importNotebook('ask llm to summerize')
const {askLlamaMatchingFunction} = importNotebook('ask llm matching function')

async function searchLlama(query, session) {
    if(!session) {
        let {getSession} = (await llama)
        session = await getSession()
    }
    // TODO: use AI to binary/boolean merge search groups of functions for the query
    let cellCache = importNotebook('cell cache').cellCache
    let functions = await askLlamaMatchingFunction(query, session)

    /*
    for(let i = 0; i < cellCache.length; i++) {
        let cell = cellCache[i]
        if(!cell[2].questions || !cell[2].questions[0]) continue
        let code = lookupCell(cell[1], cacheCells)
        break;
    }
    */

    console.log('Result: ', functions)
    // TODO: load multiple models based on the tool i'm using
    
    
    
    const q2 = "What did you just say?";
    console.log("User: " + q2);
    
    const a2 = await session.prompt(q2);
    console.log("AI: " + a2);
}

module.exports = {
    searchLlama
}



ask llm which function matches?

ask llm matching function?


In [ ]:
const importNotebook = importer.import
const llama = importNotebook('create llm session')
const {askLlamaAboutFunctions} = importNotebook('ask llm about functions')
const {askLlamaAboutCode} = importNotebook('ask llm about code')
const {askLlamaToSummerize} = importNotebook('ask llm to summerize')
const lookupCell = importer.lookupCell
const {getExports, cacheCells} = importNotebook([
  'select code tree', 'get exports from source', 'cache notebook', 'cache all'])
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { storeLlamaFunction } = importer.import('store llama function')

async function askLlamaMatchingFunction(query, session) {
  if(!session) {
    let {getSession} = (await llama)
    session = await getSession()
  }

  let cellCache = importNotebook('cell cache').cellCache
  let functions = []
  let descriptions = []

  for(let i = 0; i < cellCache.length; i++) {
    let cell = cellCache[i]
    if(!cell[2].questions || !cell[2].questions[0]) continue
    let code = lookupCell(cell[1], cacheCells)
    let summary
    let shortened
    let rpcFunction
    if(typeof functionCache[cell[1]] != 'undefined' && code.mtime <= functionCache[cell[1]].mtime) {
      summary = functionCache[cell[1]].summary
      shortened = functionCache[cell[1]].shortened
      rpcFunction = functionCache[cell[1]].rpcFunction
    } else {
      summary = await askLlamaAboutCode(code.code, session)
      shortened = await askLlamaToSummerize(summary, session)
      try {
        if(code.language == 'javascript')
          rpcFunction = getExports(code.code)
      } catch (e) {}
    }
    storeLlamaFunction(cell[1], code.mtime, rpcFunction, summary, shortened.split('-').slice(1).join('-'), shortened.split('-')[0].trim())
    if(!rpcFunction) continue
    // TODO: insert rpc function into sqlite database to make subsequent lookups faster

    functions[functions.length] = rpcFunction[0]
    descriptions[descriptions.length] = code.questions.sort((a, b) => b.length - a.length)[0]
    
    if(functions.length == 20) {
        let result = await askLlamaAboutFunctions(query, functions, descriptions, session)
        descriptions = [descriptions[functions.indexOf(result)]]
        functions = [result]
    }
  }
  if(functions.length > 1) {
      let result = await askLlamaAboutFunctions(query, functions, descriptions, session)
      functions = [result]
  }
}

module.exports = {
  askLlamaMatchingFunction
}




ask llm about functions?



In [ ]:
const importNotebook = importer.import
const llama = importNotebook('create llm session')

async function askLlamaAboutFunctions(query, functions, descriptions, session) {
  if(!session) {
    let {getSession} = (await llama)
    session = await getSession()
  }

  let q1 = "Given a list of functions:\n";
  for(let i = 0; i < functions.length; i++) {
      if(descriptions[i]) {
          q1 += functions[i] + '\n - ' + descriptions[i] + '\n'
      } else {
          q1 += functions[i] + '\n'
      }
  }
  q1 += 'which one most closely matches the query \"' + query + '\"?'
  console.log("User: " + q1);
  const a1 = await session.prompt(q1);
  console.log("AI: " + a1);
  // TODO: parse function name
  let result = a1.split(/["`*']/gi).filter(x => functions.indexOf(x) > -1)[0]
  return result
}

module.exports = {
  askLlamaAboutFunctions,
}




ask llm about code?


In [ ]:
const importNotebook = importer.import
const llama = importNotebook('create llm session')


async function askLlamaAboutCode(code, session) {
  if(!session) {
    let {getSession} = (await llama)
    session = await getSession()
  }

  const q2 = "Give me a very short description of this code?\n" + code.substr(0, 2048);
  console.log("User: " + q2);

  const a2 = await session.prompt(q2);
  console.log("AI: " + a2);

  return a2
}

module.exports = {
  askLlamaAboutCode,
}



ask llm about notebooks?



In [ ]:

const path = require('path')
const importNotebook = importer.import
const llama = importNotebook('create llm session')
const {listInProject} = importer.import('list project files');
const {askLlamaAboutFunctions} = importNotebook('ask llm about functions')

function findNotebooks(dirname) {
    return listInProject(dirname, '{,*,*/,*/*/*,*/*/*/*}*.ipynb')
}

async function askLlamaAboutNotebooks(query, session) {
  if(!session) {
    let {getSession} = (await llama)
    session = await getSession()
  }

  let notebooks = findNotebooks(path.resolve(__dirname, '../'))
  let functions = []
  let descriptions = []

  for(let i = 0; i < notebooks.length; i++) {
    let notebookName = path.basename(notebooks[i]).replace('.ipynb', '')
    let notebookCamel = notebookName.substring(0, 1).toLocaleUpperCase() + notebookName.substring(1)
    let folderName = path.basename(path.dirname(notebooks[i]))
    let folderCamel = folderName.substring(0, 1).toLocaleUpperCase() + folderName.substring(1)

    functions[functions.length] = 'search' + folderCamel.replace(/[^a-z0-9]/gi, '') + notebookCamel.replace(/[^a-z0-9]/gi, '').substring(0, 15)
    descriptions[descriptions.length] = folderCamel + ' ' + notebookCamel

    // TODO: convert this to something like function.length + description.length + 4 < 2048
    if(functions.length == 20) {
      let result = await askLlamaAboutFunctions(query, functions, descriptions, session)
      if(result) {
        descriptions = [descriptions[functions.indexOf(result)]]
        functions = [result]
      } else {
        descriptions = []
        functions = []
      }
    }
  }
  if(functions.length > 1) {
    let result = await askLlamaAboutFunctions(query, functions, descriptions, session)
    if(result) functions = [result]
    else functions = []
  }

  return functions
}


module.exports = {
  askLlamaAboutNotebooks,
}




ask llm to summerize?


In [ ]:
const importNotebook = importer.import
const llama = importNotebook('create llm session')


async function askLlamaToSummerize(query, session) {
  if(!session) {
    let {getSession} = (await llama)
    session = await getSession()
  }

  let q1 = 'Summerize this into one or two sentences:\n' + query
  console.log("User: " + q1);
  const a1 = await session.prompt(q1);
  console.log("AI: " + a1);

  let q2 = 'How would you categorize this in two or three words:\n' + a1
  console.log("User: " + q2);
  const a2 = await session.prompt(q2);
  console.log("AI: " + a2);

  return a2 + ' - ' + a1
}



module.exports = {
  askLlamaToSummerize
}





how to scan sqlite tools?





how to scan chat logs?





how to scan code history?







store llama function?




In [ ]:
var { functionCache, updateCode } = importer.import([
  'cache rpc functions with llm descriptions', 'update code cell'
])

function storeLlamaFunction (cellId, mtime, exports, description, summary, categories) {
  if(typeof functionCache != 'object') {
    functionCache = {}
  }
  functionCache[cellId] = {
    mtime,
    exports,
    description,
    summary,
    categories
  }
  var code = `
// cell cache automatically replaced
var functionCache = ${JSON.stringify(functionCache, null, 4)}

module.exports = {
  functionCache
}
`
  var cacheCell = importer.interpret('cache rpc functions with llm descriptions')
  updateCode(cacheCell, code)
}

module.exports = {
  storeLlamaFunction
}



how to scan calendar journal entries?

how to ask if my theories are correct?





create llm session?




In [ ]:
import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session

async function createSession() {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'gemma-2-9b-it-Q6_K-Q8.gguf' /* "code-llama-7b-chat.gguf"*/ /*"Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf"*/)
      });
  }

  context = await model.createContext();
  session = new LlamaChatSession({
      contextSequence: context.getSequence()
  });

  return session
}

async function getSession() {
  if(!session) {
    await createSession()
  }
  return session
}

export default {
  createSession,
  getSession,
}
